In [1]:
# -*- coding: utf-8 -*-

import requests
from bs4 import BeautifulSoup

import time
from datetime import datetime, timedelta
from tqdm import tqdm

import pandas as pd
import json

# 본 작업

In [15]:
# 석호님 csv에서 상품 ID 수집 
# (탑 리뷰어들이 리뷰를 작성한 상품들의 ID임. 개많을 예정) 
topReviews_fullver = pd.read_csv('./topReviews_fullver.csv')
goodsNo = topReviews_fullver['goodsNo'].unique() #이게 상품 ID 목록
URL_form = 'https://www.oliveyoung.co.kr/store/goods/getGdasNewListJson.do'

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
#본게임

topReviews_fullver = pd.read_csv('./topReviews_fullver.csv')
goodsNo = topReviews_fullver['goodsNo'].unique() #이게 상품 ID 목록
# 상품 ID를 가져오고, 페이지는 while로 날짜 단위 순환시킨 다음 페이지 번호 변경

def col_isNaN(cols):
    return True if cols != 10 else False

def product_reviews_of_topReviewer(end_date):
    page = 1
    df = pd.DataFrame()
    
    data = {
            "goodsNo": "A000000180332", 
            "gdasSort": "02", 
            "itemNo": "all_search", 
            "pageIdx": 1, 
            "colData": '', 
            "keywordGdasSeqs": '', 
            "type": '', 
            "point": '', 
            "hashTag": '', 
            "optionValue": '', 
            "cTypeLength": 0
            }
    
    idx = 1
    for goodNo in goodsNo:
        # good_No_idx = goodNo
        data['goodsNo'] = goodNo
        page = 1
        
        while True:
            data["pageIdx"] = page
            response = requests.get(URL_form, 
                                    headers = {'X-Requested-With': 'XMLHttpRequest'},
                                    params = data
                                    )
            data_dict = response.json()
            #리뷰 데이터 자체가 없는 경우
            if not data_dict:
                # print(idx, goodNo, "ERROR1")
                break
            df_add = pd.DataFrame.from_dict(data_dict['gdasList'])
            isNaN = df_add.isna().sum().apply(col_isNaN)
            
            use_col = []
            for idx, col in enumerate(df_add.columns):
                if isNaN[idx]:
                    use_col.append(col)
            df_add =  df_add[use_col]
            
            # 리뷰 데이터를 받아왔는데, 실제로는 리뷰가 없는 경우
            if not len(df_add): 
                # print(idx, goodNo, "ERROR2")
                break
            
            # 새로운 페이지의 리뷰를 concat 예정
            # 그 전에, 해당 페이지의 10개 리뷰 중 날짜 조건에 해당하는 리뷰까지만 가져올 예정
            # 어차피 두 달 정도는 탐색을 해야되기 때문에 매 페이지마다 모든 행을 다 검사하지는 않고
            # 마지막 행을 확인 후 2달 넘어가는 페이지일 경우에만 모든 행 검사 예정
            
            # #마지막 페이지가 될지 검사해보자
            # # 새로 가져온 페이지의 마지막 리뷰 날짜 확인
            
            date_obj = datetime.strptime(df_add.iloc[-1]['dispRegDate'], '%Y.%m.%d')
            if (end_date - date_obj).days > 60:
                # print(idx, goodNo, "60일이후리뷰")
                break

            #검사 끝, 병합 진행
            df = pd.concat([df, df_add])
            # 다음 페이지로 인덱스 변환
            page +=1
        
        print("현재", list(goodsNo).index(goodNo), "번째 상품까지 진행됨.")    
        if list(goodsNo).index(goodNo) % 1000 == 0:
            temp_result = df
            temp_result.to_csv(f'temp_result_{list(goodsNo).index(goodNo) // 1000}.csv', sep = ',', escapechar = ',', index=False)
            
    return df.reset_index(drop=True)
                
end_date = datetime(2023,4,21)
result = product_reviews_of_topReviewer(end_date)




# gds_count : goodsNo[goodsNo.index[-1]] 전체를 돌리려면 마지막 인덱스를 넣어야 하니까

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


현재 0 번째 상품까지 진행됨.
현재 1 번째 상품까지 진행됨.
현재 2 번째 상품까지 진행됨.
현재 3 번째 상품까지 진행됨.
현재 4 번째 상품까지 진행됨.
현재 5 번째 상품까지 진행됨.
현재 6 번째 상품까지 진행됨.
현재 7 번째 상품까지 진행됨.
현재 8 번째 상품까지 진행됨.
현재 9 번째 상품까지 진행됨.
현재 10 번째 상품까지 진행됨.
현재 11 번째 상품까지 진행됨.
현재 12 번째 상품까지 진행됨.
현재 13 번째 상품까지 진행됨.
현재 14 번째 상품까지 진행됨.
현재 15 번째 상품까지 진행됨.
현재 16 번째 상품까지 진행됨.
현재 17 번째 상품까지 진행됨.
현재 18 번째 상품까지 진행됨.
현재 19 번째 상품까지 진행됨.
현재 20 번째 상품까지 진행됨.
현재 21 번째 상품까지 진행됨.
현재 22 번째 상품까지 진행됨.
현재 23 번째 상품까지 진행됨.
현재 24 번째 상품까지 진행됨.
현재 25 번째 상품까지 진행됨.
현재 26 번째 상품까지 진행됨.
현재 27 번째 상품까지 진행됨.
현재 28 번째 상품까지 진행됨.
현재 29 번째 상품까지 진행됨.
현재 30 번째 상품까지 진행됨.
현재 31 번째 상품까지 진행됨.
현재 32 번째 상품까지 진행됨.
현재 33 번째 상품까지 진행됨.
현재 34 번째 상품까지 진행됨.
현재 35 번째 상품까지 진행됨.
현재 36 번째 상품까지 진행됨.
현재 37 번째 상품까지 진행됨.
현재 38 번째 상품까지 진행됨.
현재 39 번째 상품까지 진행됨.
현재 40 번째 상품까지 진행됨.
현재 41 번째 상품까지 진행됨.
현재 42 번째 상품까지 진행됨.
현재 43 번째 상품까지 진행됨.
현재 44 번째 상품까지 진행됨.
현재 45 번째 상품까지 진행됨.
현재 46 번째 상품까지 진행됨.
현재 47 번째 상품까지 진행됨.
현재 48 번째 상품까지 진행됨.
현재 49 번째 상품까지 진행됨.
현재 50 번째 상품까지 진행됨.
현재 51 번째 상품까지 진행됨.
현재 52 번째 상품까지 진행됨.
현재 

최종 결과 파일로 변환

In [ ]:
result.to_csv('product_reviews.csv', sep = ',', escapechar = ',', index=False)

In [4]:
temp_0 = pd.read_csv('temp_result_0.csv')
len(temp_0['goodsNo'].unique())

3